In [ ]:
pip install spotipy


In [1]:
import pandas as pd
import numpy as np
import spotipy
import json
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
CLIENT_ID = "a14dd1fdde5846c2b1086730d6da1f44"
CLIENT_SECRET = "4e6dd4a75d654f939c451be63e7e53cb"

In [3]:
token = spotipy.oauth2.SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
cache_token = token.get_access_token()
sp = spotipy.Spotify(cache_token)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  


In [ ]:
sp.user_playlist_tracks("spotify", "37i9dQZF1DX7Jl5KP2eZaS")

In [4]:
def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature","acousticness"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        playlist_features["rank"] = track
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0] #gets features such as danceability, energy ,etc
        for feature in playlist_features_list[4:]: #adds the feature values to the df for the matching columns
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
    
    #Assigning rank to the songs 
    playlist_df['rank'] = np.arange(len(playlist_df)) +1     
    return playlist_df

In [ ]:
track_20 = analyze_playlist("spotify", "37i9dQZF1DX7Jl5KP2eZaS")
track_19 = analyze_playlist("spotify", "37i9dQZF1DXcz8eC5kMSWZ")
track_18 = analyze_playlist("spotify", "37i9dQZF1DX1HUbZS4LEyL")

In [ ]:
track_20

In [5]:
playlist_dict = {
 "top_2020" : ("spotify", "37i9dQZF1DX7Jl5KP2eZaS"), #not released yet
 "top_2019" : ("spotify", "37i9dQZF1DWVRSukIED0e9"), #100
 "top_2018" : ("spotify", "37i9dQZF1DXe2bobNYDtW8"), #100 songs 
 "top_2017" : ("spotify", "37i9dQZF1DWTE7dVUebpUW"),  #100 songs 
 "top_2016" : ("spotify", "37i9dQZF1DX8XZ6AUo9R4R"),  #100 songs 
 "top_2015" : ("spotify", "37i9dQZF1DX9ukdrXQLJGZ"),  #100 songs 
 "top_2014" : ("spotify", "37i9dQZF1DX0h0QnLkMBl4"), #100 songs 
 "top_2013" : ("spotify", "37i9dQZF1DX3Sp0P28SIer"), #100 songs 
 "top_2012" : ("spotify", "37i9dQZF1DX0yEZaMOXna3"), #100 songs 
 "top_2011" : ("spotify", "37i9dQZF1DXcagnSNtrGuJ"),#100 songs 
 "top_2010" : ("spotify", "37i9dQZF1DXc6IFF23C9jj"),  #100 songs 
 "top_2009" : ("spotify", "37i9dQZF1DX4UkKv8ED8jp"),  #100 songs 
 "top_2008" : ("spotify", "37i9dQZF1DWYuGZUE4XQXm"),  #100 songs 
 "top_2007" : ("spotify", "37i9dQZF1DX3j9EYdzv2N9"), #100 songs 
 "top_2006" : ("spotify", "37i9dQZF1DX1vSJnMeoy3V"), #100 songs 
 "top_2005" : ("spotify", "37i9dQZF1DWWzQTBs5BHX9"), #100 songs 
 "top_2004" : ("spotify", "37i9dQZF1DWTWdbR13PQYH"),#100 songs 
 "top_2003" : ("spotify", "37i9dQZF1DXaW8fzPh9b08"), #100 songs 
 "top_2002" : ("spotify", "37i9dQZF1DX0P7PzzKwEKl"), #100 songs 
 "top_2001" : ("spotify", "37i9dQZF1DX9Ol4tZWPH6V"), #100 songs 
 "top_2000" : ("spotify", "37i9dQZF1DWUZv12GM5cFk")#100 songs 
  
    
    
}

In [ ]:
for i, (key, val) in enumerate(playlist_dict.items()):
    print(*val)

In [6]:
def analyze_playlist_dict(playlist_dict):
    
    # Loop through every playlist in the dict and analyze it
    for i, (key, val) in enumerate(playlist_dict.items()):
        playlist_df = analyze_playlist(*val)
        # Add a playlist column so that we can see which playlist a track belongs too
        playlist_df["playlist"] = key
        # Create or concat df
        if i == 0:
            playlist_dict_df = playlist_df
        else:
            playlist_dict_df = pd.concat([playlist_dict_df, playlist_df], ignore_index = True)
            
    return playlist_dict_df

In [7]:
multiple_playlist_df = analyze_playlist_dict(playlist_dict)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [8]:
multiple_playlist_df["playlist"].count()

2050

In [9]:
multiple_playlist_df.to_excel(r'D:\dektop data\spotify project\spotify_data.xlsx', index = False)